<a href="https://colab.research.google.com/github/Computer-CGuy/etherpaisahipaisa/blob/master/Ether_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --quiet cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

In [3]:
import torch_xla
import torch_xla.core.xla_model as xm

In [4]:
# CORES = len(torch_xla.core.xla_model.get_xla_supported_devices())
# print("Number of cores",CORES)

In [5]:
import torch
import time

In [6]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = xm.xla_device()
print("Using",str(device),"as your PyTorch Device")

Using xla:1 as your PyTorch Device


In [7]:
ROUNDS = 24
dtype = torch.long

def ROL64(a,offset):
    # p = torch.clone(a)
    # # print(offset[1].shape)
    # # print(p.shape)
    # p/=(offset[1])
    return ((a*offset[0]) ^ (a/offset[1]).long())

def tu_ROL64(a,offset):
    p = torch.clone(a)
    print(offset[:,1].shape)
    print(p.shape)
    p/=(offset[:,1])
    return ((a*offset[:,0]) ^ p)
def index(x,y):
    return (((x)%5)+5*((y)%5))

In [8]:
KeccakRoundConstants = torch.zeros((25), dtype=dtype).to(device)

one = torch.LongTensor([1,2]).to(device)
# KeccakRhoOffsets = torch.LongTensor([0 , 1 , 62, 28, 27, 36, 44, 6, 55, 20, 3, 10,43, 25, 39, 41, 45, 15, 21, 8, 18, 2, 61, 56, 14]).to(device)
KeccakRhoOffsets = torch.ones([2,25,1],dtype=dtype).to(device)

# For pi

shift_pi = torch.zeros([25,25],dtype=torch.int64).to(device)
l =[0,6,12,18,24,3,9,10,16,22,1,7,13,19,20,4,5,11,17,23,2,8,14,15,21]
for g in range(25):
	shift_pi[g][l[g]]=1
# shift_pi = shift_pi.to(device)

In [9]:
# state = torch.zeros([25,batch],dtype=dtype)
shift_pi= shift_pi.unsqueeze(2)

In [13]:
def theta(A,C,D):
    C[:] = 0
    for x in range(5):
      C[x]^=A[5*x:(x+1)*5]

    #   for y in range(5):
    #       C[x] ^= A[index(x, y)]
    # torch.bitwise_xor(A[0::5]^A[1::5]^A[2::5]^A[3::5],A[4::5],out=C)
    D = ROL64(C, one)
    # print(C.shape)
    # print(D.shape)
    # print(A.shape)
    for x in range(5):
        # A[x,x+5]
        for y in range(5):
            # print(A[index(x, y)].shape)
            # print(D[(x+1)%5].shape)
            # print(C[(x+4)%5].shape)
            A[index(x, y)] = A[index(x, y)]^D[(x+1)%5] ^ C[(x+4)%5]
        # A[x::5] = A[x::5]^D[(x+1)%5] ^ C[(x+4)%5]

def rho(A,C,D):
    # for x in range(5):
    #   for y in range(5):
    A = ROL64(A, KeccakRhoOffsets)

def pi(A,C,D):
    # (A*shift_pi)
    # torch.sum((A*shift_pi),axis=1,out=A)
    # temp = torch.clone(A)
    # for x in range(5):
    #   for y in range(5):
    #       A[index(0*x+1*y, 2*x+3*y)] = temp[index(x, y)]
    # torch.tensordot(shift_pi,A,dims=1,out=A)
    ret = (A.unsqueeze(1)*shift_pi)
    A = ret[0]
    for x in range(1,ROUNDS):
        A+=ret[x]
def chi(A,C,D):
    for y in range(5):
        for x in range(5):
            C[x] = A[index(x, y)] ^ ((~A[index(x+1, y)]) & A[index(x+2, y)])
        for x in range(5):
            A[index(x, y)] = C[x]

def iota(A,indexRound):
    A[index(0, 0)] ^= KeccakRoundConstants[indexRound]

def mine(state,C,D):
    for i in range(ROUNDS):
        theta(state,C,D)
        rho(state,C,D)
        pi(state,C,D)
        chi(state,C,D)
        iota(state, i)

In [ ]:
def mp_fn(index):
    global batch
    global t0
    state = torch.zeros([25,batch],dtype=dtype).to(device)
    C = torch.zeros([5,batch],dtype=dtype).to(device)
    # C = torch.zeros([25,batch],dtype=dtype).to(device)
    D = torch.clone(C)
    # D = torch.zeros([25,batch],dtype=dtype).to(device)
    t0= time.time()
    mine(state, C, D)
    print(state.shape)
batch = 3000000
t0 = time.time()

print(f"{batch:,}")
mp_fn(0)

t1 = time.time() - t0
t1/=batch
print(f"{1/t1:,}")

3,000,000


In [9]:
import  torch_xla.debug.metrics as met

print(met.metrics_report())

Metric: IrValueTensorToXlaData
  TotalSamples: 4
  Accumulator: 051ms466.171us
  ValueRate: 03s947ms296.449us / second
  Rate: 229.067 / second
  Percentiles: 1%=005ms847.540us; 5%=005ms847.540us; 10%=005ms847.540us; 20%=005ms847.540us; 50%=005ms371.505us; 80%=036ms351.376us; 90%=036ms351.376us; 95%=036ms351.376us; 99%=036ms351.376us
Metric: OutboundData
  TotalSamples: 4
  Accumulator: 5.48KB
  ValueRate: 226.38KB / second
  Rate: 165.105 / second
  Percentiles: 1%=16.00B; 5%=16.00B; 10%=16.00B; 20%=16.00B; 50%=400.00B; 80%=4.88KB; 90%=4.88KB; 95%=4.88KB; 99%=4.88KB
Metric: TransferToServerTime
  TotalSamples: 4
  Accumulator: 051ms386.220us
  ValueRate: 03s939ms737.485us / second
  Rate: 228.757 / second
  Percentiles: 1%=005ms837.806us; 5%=005ms837.806us; 10%=005ms837.806us; 20%=005ms837.806us; 50%=005ms360.134us; 80%=036ms302.333us; 90%=036ms302.333us; 95%=036ms302.333us; 99%=036ms302.333us
Metric: TransferToServerTransformTime
  TotalSamples: 4
  Accumulator: 022ms353.688us
  Valu